## Frequenza di funzionamento PS e PL

In [1]:
from pynq import Clocks

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')
print(f'FCLK2: {Clocks.fclk2_mhz:.6f}MHz')
print(f'FCLK3: {Clocks.fclk3_mhz:.6f}MHz')


CPU:   650.000000MHz
FCLK0: 100.000000MHz
FCLK1: 100.000000MHz
FCLK2: 100.000000MHz
FCLK3: 100.000000MHz


## Driver: MySha256 Class

Versione che segue le linee guida della [documentazione PYNQ.](https://pynq.readthedocs.io/en/v2.7.0/overlay_design_methodology/overlay_tutorial.html)

In [2]:
from pynq import DefaultIP
from pynq import allocate
import time
import numpy as np
import mmap
import cProfile as profile
import warnings
warnings.filterwarnings('ignore')


class StreamSha256IP(DefaultIP): 
    
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['rsl:hls:sha256:1.2']

from pynq import DefaultHierarchy
from pynq import allocate
import time
import numpy as np

class StreamSha256Driver(DefaultHierarchy):
    
    def __init__(self, description):
        super().__init__(description)
        self.size=65536 #pow(2,16)
        self.input_buffer=allocate(shape=self.size, dtype=np.uint8)
        self.out_buffer=allocate(shape=32, dtype=np.uint8)
        self.send=self.axi_dma.sendchannel
        self.recv=self.axi_dma.recvchannel
        
    
    def __del__(self):
        del self.input_buffer, self.out_buffer
    
    def resizeInputBuffer(self, size):
        del self.input_buffer
        self.size=size
        self.input_buffer=allocate(shape=self.size, dtype=np.uint8)
    
    def hashOfBigFile(self, data):
        #t_start = time.time()
        #data=np.memmap(file_path, dtype=np.uint8)
        
        BUF_SIZE=(self.size-1)
        
        leftover=len(data)
        
        #_from=0
        _to= BUF_SIZE if leftover>BUF_SIZE else leftover
        leftover-=_to

        self.input_buffer[0]=1
        
        i=0
        offset=BUF_SIZE*i
        
        #print(t_stop-t_start)
        
        while True:
            
            #print(offset)
            if _to!=0:
                self.input_buffer[1:_to+1] = data[offset:offset+_to]
                self.send.transfer(self.input_buffer, nbytes=_to+1)
                self.send.wait()
            else:
                self.input_buffer[0]=0
                self.send.transfer(self.input_buffer, nbytes=1)
                self.send.wait()
                break;
            #print(self.input_buffer[0:_to+1]) 
            _to= BUF_SIZE if leftover>BUF_SIZE else leftover
            leftover-=_to
            i=i+1
            offset=BUF_SIZE*i
            
            
        self.recv.transfer(self.out_buffer)
        self.recv.wait()
        #t_stop = time.time()
        stringa="".join(["{:02X}".format(item) for item in self.out_buffer])
    
        #file.close()            
        #del source_data
        #print(t_stop-t_start)
        return stringa
    
    
    def startSha(self):
        self.sha256_0.write(0x0, 0x81)
    

    @staticmethod
    def checkhierarchy(description):
        #print('sha256_0' in description["ip"])
        if 'axi_dma' in description["ip"] and 'sha256_0' in description["ip"]:
            check= True
        else:
            check= False
        #print(check)
        return check
    
from pynq import Overlay

class TestOverlay(Overlay):
    def __init__(self, bitfile, **kwargs):
        super().__init__(bitfile, **kwargs)

    #def hashOf(self, data):
    #    return self.sha256.hashOfString(data)
    
    #def hashOfFile(self, data):
    #    return self.sha256.hashOfFile_(data)
    
    def hashOfBigFile(self, data):
        return self.sha256.hashOfBigFile(data)
    
    def start(self):
        return self.sha256.startSha()
    
    def resize(self, size):
        self.sha256.resizeInputBuffer(size)
        
    def bar(self, data):
        profile.runctx('self.hashOfBigFile(data)', globals(), locals())

ol = TestOverlay("example_design.bit")
ol.start()

## Driver: MySha256 Function

Versione che definisce direttamente una funzione, senza utilizzare la classe, alleggerendo in questo modo lo stack

In [3]:
#HASH OF
from pynq import DefaultIP
from pynq import allocate
import time
import numpy as np
import mmap
import cProfile as profile

from pynq import DefaultHierarchy
from pynq import allocate
from pynq import Overlay
import time
import numpy as np

if 'input_buffer' in locals():
    del input_buffer

if 'out_buffer' in locals():
    del out_buffer


if 'buffer' in locals():
    del buffer[1], buffer[0]
    
size = pow(2,16)
    
    
buffer=[None]*2
buffer[0]=allocate(shape=(size,), dtype=np.uint8)
buffer[1]=allocate(shape=(size,), dtype=np.uint8)
    
    
#ol = Overlay("example_design.bit")
ol.sha256.sha256_0.write(0x0, 0x81)
sha256_dma=ol.sha256.axi_dma
send=sha256_dma.sendchannel
recv=sha256_dma.recvchannel

input_buffer=allocate(shape=(size,), dtype=np.uint8)
out_buffer=allocate(shape=(32,), dtype=np.uint8) 

def hashOf(data):
    #t_start = time.time()        
    
    BUF_SIZE=(size-1)
        
    leftover=len(data)
    _to= BUF_SIZE if leftover>BUF_SIZE else leftover
    leftover-=_to
        
    i=0
    offset=BUF_SIZE*i
    input_buffer[0]=1
    #t_start = time.time()
    while True:
        if _to!=0:
                input_buffer[1:_to+1] = data[offset:offset+_to]
                send.transfer(input_buffer, nbytes=_to+1)
                send.wait()
        else:
                input_buffer[0]=0
                send.transfer(input_buffer, nbytes=1)
                send.wait()
                break;
            #print(self.input_buffer[0:_to+1]) 
        _to= BUF_SIZE if leftover>BUF_SIZE else leftover
        leftover-=_to
        i=i+1
        offset=BUF_SIZE*i
            
    recv.transfer(out_buffer)
    recv.wait()
    #t_stop = time.time()
    stringa="".join(["{:02X}".format(item) for item in out_buffer])

    #print(t_stop-t_start)
    return stringa

## Driver: MySha256 Function with 2 Buffer

Versione che sfrutta 2 buffer nella lettura, in modo che mentre un buffer è impegnato nell'invio dei dati da DRAM a DMA l'altro può leggere un secondo blocco, alternandosi fino al completamento della lettura del file.

In [4]:
def hashOf_2buffer(data):
    #t_start = time.time()        
    
    BUF_SIZE=(size-1)
    
    ch=0
        
    leftover=len(data)
    _to= BUF_SIZE if leftover>BUF_SIZE else leftover
    leftover-=_to
        
    i=0
    offset=BUF_SIZE*i
    buffer[0][0]=1
    buffer[1][0]=1
    #t_start = time.time()
    while True:
        if _to!=0:
            buffer[ch][1:_to+1] = (data[offset:offset+_to])
            send.wait()
            send.transfer(buffer[ch], nbytes=_to+1)
            ch=(ch+1)%2 
        else:
            buffer[ch][0]=0
            send.wait()
            send.transfer(buffer[ch], nbytes=1)
            send.wait()
            break;
            #print(self.input_buffer[0:_to+1]) 
        _to= BUF_SIZE if leftover>BUF_SIZE else leftover
        leftover-=_to
        i=i+1
        offset=BUF_SIZE*i
            
    recv.transfer(out_buffer)
    recv.wait()
    #t_stop = time.time()
    stringa="".join(["{:02X}".format(item) for item in out_buffer])

    #print(t_stop-t_start)
    return stringa

## Test:

In [35]:
import hashlib
import os

file="/home/xilinx/jupyter_notebooks/Progetto/sampleFile/sample.txt"


m = hashlib.sha256()

ol = TestOverlay("example_design.bit")
ol.start()

print("file size: " + str(os.stat(file).st_size) )
print()

source_data = np.memmap(file, dtype=np.uint8)

t_start = time.time()
m.update(source_data)
digest="".join(["{:02X}".format(item) for item in m.digest()])
t_stop = time.time()
print("hashlib:\t"+str(t_stop-t_start)+"\thash: "+digest)

t_start = time.time()
digest0=ol.sha256.hashOfBigFile(source_data)
t_stop = time.time()
print("mySha256 class:\t"+str(t_stop-t_start)+"\thash: "+digest0)

t_start = time.time()
digest1=hashOf(source_data)
t_stop = time.time()
print("mySha256 fun:\t"+str(t_stop-t_start)+"\thash: "+digest1)

t_start = time.time()
digest2=hashOf_2buffer(source_data)
t_stop = time.time()
print("mySha256 2buf:\t"+str(t_stop-t_start)+"\thash: "+digest2)

print()
print("Driver C:")
#!ls -l /home/xilinx/jupyter_notebooks/Progetto/DriverC/sha256/build/
#!chmod 777 /home/xilinx/jupyter_notebooks/Progetto/DriverC/sha256/build/sha256
!/home/xilinx/jupyter_notebooks/Progetto/DriverC/sha256/build/sha256 {file}

del source_data


file size: 536870896

hashlib:	23.848477840423584	hash: 21BE956416A1669FA79B498827FB4D3F24F3CA4DA611523459C5166518459B4F
mySha256 class:	24.767296314239502	hash: 21BE956416A1669FA79B498827FB4D3F24F3CA4DA611523459C5166518459B4F
mySha256 fun:	24.62043070793152	hash: 21BE956416A1669FA79B498827FB4D3F24F3CA4DA611523459C5166518459B4F
mySha256 2buf:	24.60295295715332	hash: 21BE956416A1669FA79B498827FB4D3F24F3CA4DA611523459C5166518459B4F

Driver C:
producer time:	23.548015
consumer time:	6.893888
total time:	23.850435
21 BE 95 64 16 A1 66 9F A7 9B 49 88 27 FB 4D 3F 24 F3 CA 4D A6 11 52 34 59 C5 16 65 18 45 9B 4F 

